In [1]:
%pip install --upgrade --quiet langchain langchain-community langchain-openai langchain-experimental neo4j tiktoken python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install --upgrade pymupdf

In [5]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader, OnlinePDFLoader, PyPDFium2Loader, PDFMinerLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough

/Users/williamzebrowski/Library/Mobile Documents/com~apple~CloudDocs/transform_llm/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [19]:
# import dotenv
# import os
# from neo4j import GraphDatabase

# load_status = dotenv.load_dotenv("Neo4j-5ab14197-Created-2024-08-06.txt")
# if load_status is False:
#     raise RuntimeError('Environment variables not loaded.')

# URI = os.getenv("NEO4J_URI")
# AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

# with GraphDatabase.driver(URI, auth=AUTH) as driver:
#     driver.verify_connectivity()
#     print("Connection established.")

ValueError: Cannot resolve address df5e5422.databases.neo4j.io:7687

In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["NEO4J_URI"] = "neo4j+s://5ab14197.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "MewfGiGycd1Q3j8bE_Ej_U5z4exEw9oFenVfNYu8Cdo"
os.environ['OPENAI_API_KEY']

graph = Neo4jGraph()

# Data Ingestion

In [36]:
# raw_documents = PyMuPDFLoader('/Users/williamzebrowski/Library/Mobile Documents/com~apple~CloudDocs/transform_llm/rag/AIAYN.pdf')
raw_documents = PyMuPDFLoader("/Users/williamzebrowski/Library/Mobile Documents/com~apple~CloudDocs/transform_llm/rag/knowledge_graphs/William-Zebrowski-Resume-AI-Eng.pdf")

# pages[0]

In [37]:
data = raw_documents.load()

In [39]:
data[1]

Document(metadata={'source': '/Users/williamzebrowski/Library/Mobile Documents/com~apple~CloudDocs/transform_llm/rag/knowledge_graphs/William-Zebrowski-Resume-AI-Eng.pdf', 'file_path': '/Users/williamzebrowski/Library/Mobile Documents/com~apple~CloudDocs/transform_llm/rag/knowledge_graphs/William-Zebrowski-Resume-AI-Eng.pdf', 'page': 1, 'total_pages': 3, 'format': 'PDF 1.4', 'title': 'Resume Viewer', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Chromium', 'producer': 'Skia/PDF m123', 'creationDate': "D:20240801151953+00'00'", 'modDate': "D:20240801151953+00'00'", 'trapped': ''}, page_content='• Worked with product managers & design teams to develop new product experiences using NLP & LLMs.\n• Defined and improved business & product metrics to optimize the quality and cost of AI usage.\n• Advocated for, and built, new and better ways of doing things, ensuring continuous improvement in every project.\nMAR 2023 - JUL 2023\nAutomation Engineer, Benefits Data Trust, Philadelphia

In [40]:
# Read the wikipedia article
# raw_documents = WikipediaLoader(query="Elizabeth I").load()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=10)
documents = text_splitter.split_documents(data)

In [41]:
llm=ChatOpenAI(temperature=0, model_name="gpt-4o-mini") # gpt-4-0125-preview occasionally has issues
llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [42]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [27]:
# Retriever

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [43]:
entity_chain.invoke({"question": "whose resume is this?"}).names

['resume']

In [44]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    print(f'generate full text query: {full_text_query.strip()}')
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
        print(f"structured_retriever result: {result}")
    return result

In [45]:
print(structured_retriever("who is the author?"))

generate full text query: author~2
structured_retriever result: 



In [46]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [47]:
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [48]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [50]:
chain.invoke({"question": "Who is the AI Engineer?"})

Search query: Who is the AI Engineer?
generate full text query: AI~2 AND Engineer~2
structured_retriever result: Ai Engineer - EMPLOYED_BY -> Benefits Data Trust
Ai Engineer - WORKED_ON -> Generative Ai Development
Qa Engineer - WORKED_AT -> Benefits Data Trust
Python - HAS_SKILL -> Qa Engineer


'The AI Engineer is William Zebrowski, who is currently employed at Benefits Data Trust and specializes in generative AI development.'

In [51]:
# chain.invoke(
#     {
#         "question": "what are some fafsa links?",
#         "chat_history": [("What is fafsa?", "FAFSA (Free Application for Federal Student Aid) is an application that requires an FSA ID for completion. ")],
#     }
# )

In [126]:
chain.invoke({"question": "do i need my parents income information?"})

Search query: do i need my parents income information?
generate full text query: parents~2
structured_retriever result: Parents - APPLY_FOR -> Direct Unsubsidized Loan
Parents - PAY_FOR -> College
Parents - PROVIDE -> Aid Sources
Parents - DOES_NOT_AFFECT -> Financial Aid Eligibility
Parents - MUST_ENTER_INFORMATION -> Fafsa
Parents - MUST_CREATE -> Fsa Id
Parent'S Account - PROVIDE -> Month And Day Of Birth
Parent'S Account - ANSWER -> Challenge Questions
Parent'S Account - RECEIVE_CODE -> Email
Parent'S Account - RESET -> Password
Parent'S Account - RECEIVE_CODE -> Phone Number
Parent'S Account - ACCESS -> Https://Fafsa.Gov
Guidance - FOLLOW -> Parents
Https://Studentaid.Gov/Help/Unlock-Fsa-Id - INSTRUCTIONS -> Parent'S Account


"Yes, you will need your parents' income information if you are including their details on the FAFSA. However, if your parent consents to the IRS sharing their federal tax information, they won't need to answer questions about their income directly."